In [1]:
import pandas as pd
titanic_data = sqlContext.read.load('/FileStore/tables/rms_train.csv',
                                  format='com.databricks.spark.csv',
                                  header='true',
                                  inferSchema='true')
pd.DataFrame(titanic_data.take(3), columns = titanic_data.columns)

In [2]:
titanic_data.describe().toPandas().transpose()

In [3]:
###One-Hot Encoding
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

categoricalColumns = ["Sex", "Embarked"]
stages = [] # stages in our Pipeline
for categoricalCol in categoricalColumns:
  # Category Indexing with StringIndexer
  stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
  # Use OneHotEncoder to convert categorical variables into binary SparseVectors
  encoder = OneHotEncoder(inputCol=categoricalCol+"Index", outputCol=categoricalCol+"classVec")
  # Add stages.  These are not run here, but will run all at once later on.
  stages += [stringIndexer, encoder]

In [4]:
# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol = "Survived", outputCol = "label")
stages += [label_stringIdx]

In [5]:
# Transform all features into a vector using VectorAssembler
numericCols = ["Pclass", "Age", "SibSp", "Parch", "Fare"]
assemblerInputs = map(lambda c: c + "classVec", categoricalColumns) + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [6]:
cols = titanic_data.columns
# Create a Pipeline.
pipeline = Pipeline(stages=stages)
# Run the feature transformations.
#  - fit() computes feature statistics as needed.
#  - transform() actually transforms the features.
pipelineModel = pipeline.fit(titanic_data)
dataset = pipelineModel.transform(titanic_data)

# Keep relevant columns
selectedcols = ["label", "features"] + cols
dataset = dataset.select(selectedcols)
#display(dataset)
#type(dataset)
dataset.toPandas()

In [7]:
### Randomly split data into training and test sets. set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print trainingData.count()
print testData.count()

In [8]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=17)

# Train model with Training Data
lrModel = lr.fit(trainingData)

In [9]:
# Make predictions on test data using the transform() method.
# LogisticRegression.transform() will only use the 'features' column.
predictions = lrModel.transform(testData)
predictions.printSchema()

In [10]:
# View model's predictions and probabilities of each prediction class
# You can select any columns in the above schema to view as well. For example's sake we will choose age & occupation
selected = predictions.select("label", "prediction", "probability", "Age", "Fare","Pclass", "Sex", "SibSp","Parch","Embarked")
selected.toPandas()

In [11]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

In [12]:
#what is the name of the evaluator?
evaluator.getMetricName()

In [13]:
#Now we will try tuning the model with the ParamGridBuilder and the CrossValidator.
#If you are unsure what params are available for tuning, you can use explainParams()
#to print a list of all params and their definitions.
print lr.explainParams()

In [14]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())

# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(trainingData)
# this will likely take a fair amount of time because of the amount of models that we're creating and testing


In [15]:
# Use test set here so we can measure the accuracy of our model on new data

predictions = cvModel.transform(testData)
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

In [16]:
print 'Model Intercept: ', cvModel.bestModel.intercept

In [17]:
#weights = cvModel.bestModel.weights
# on Spark 2.X weights are available as ceofficients
weights = cvModel.bestModel.coefficients
weights = map(lambda w: (float(w),), weights)  # convert numpy type to float, and to tuple
weightsDF = sqlContext.createDataFrame(weights, ["Feature Weight"])
weightsDF.toPandas()

In [18]:
# View best model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "Age", "Fare","Pclass", "Sex", "SibSp","Parch","Embarked")
selected.toPandas()

In [19]:
from pyspark.ml.classification import DecisionTreeClassifier

# Create initial Decision Tree Model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=8)

# Train model with Training Data
dtModel = dt.fit(trainingData)

In [20]:
print "numNodes = ", dtModel.numNodes
print "depth = ", dtModel.depth

In [21]:
# Make predictions on test data using the Transformer.transform() method.
predictions = dtModel.transform(testData)

In [22]:
predictions.printSchema()

In [23]:
# View model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "Age", "Fare","Pclass", "Sex", "SibSp","Parch","Embarked")
selected.show()

In [24]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

In [25]:
dt.getImpurity()

In [26]:
# Create ParamGrid for Cross Validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [1,2,6,10])
             .addGrid(dt.maxBins, [20,40,80])
             .build())

In [27]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=dt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(trainingData)
# Takes ~5 minutes

In [28]:
print "numNodes = ", cvModel.bestModel.numNodes
print "depth = ", cvModel.bestModel.depth

In [29]:
# Use test set here so we can measure the accuracy of our model on new data
predictions = cvModel.transform(testData)

In [30]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

In [31]:
# View Best model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "Age", "Fare","Pclass", "Sex", "SibSp","Parch","Embarked")
selected.show()

In [32]:
from pyspark.ml.classification import RandomForestClassifier

# Create an initial RandomForest model.
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

# Train model with Training Data
rfModel = rf.fit(trainingData)

In [33]:
# Make predictions on test data using the Transformer.transform() method.
predictions = rfModel.transform(testData)

In [34]:
predictions.printSchema()

In [35]:
# View model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "Age", "Fare","Pclass", "Sex", "SibSp","Parch","Embarked")
selected.show()

In [36]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

In [37]:
# Create ParamGrid for Cross Validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [2, 4, 6])
             .addGrid(rf.maxBins, [20, 60])
             .addGrid(rf.numTrees, [5, 20])
             .build())

In [38]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations.  This can take about 6 minutes since it is training over 20 trees!
cvModel = cv.fit(trainingData)

In [39]:
# Use test set here so we can measure the accuracy of our model on new data
predictions = cvModel.transform(testData)

In [40]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

In [41]:
# View Best model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "Age", "Fare","Pclass", "Sex", "SibSp","Parch","Embarked")
selected.show()

In [42]:
bestModel = cvModel.bestModel
# Generate predictions for entire dataset
finalPredictions = bestModel.transform(dataset)
# Evaluate best model
evaluator.evaluate(finalPredictions)

In [43]:
#from pyspark.ml.classification import MultilayerPerceptronClassifier

#layers = [4, 5, 4, 3]
# Create an initial MultilayerPerceptronClassifier model.
#rf = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

# Train model with Training Data
#rfModel = rf.fit(trainingData)

In [44]:
# Use test set here so we can measure the accuracy of our model on new data
#predictions = cvModel.transform(testData)

In [45]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
#from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
#mlpc_acc=evaluator.evaluate(predictions)
#evaluator.evaluate(predictions)

In [46]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

# Create an initial RandomForest model.
layers = [8, 5, 4, 2]
mp = MultilayerPerceptronClassifier(labelCol="label", featuresCol="features", maxIter=100, layers=layers, blockSize=128, seed=1350)

# Train model with Training Data
mpModel = mp.fit(trainingData)

In [47]:
# Make predictions on test data using the Transformer.transform() method.
predictions = mpModel.transform(testData)

In [48]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Evaluate model
evaluator = MulticlassClassificationEvaluator()
mp_result = evaluator.evaluate(predictions)
print mp_result